# 1. Importing Necessary Packages

In [76]:
import numpy as np
import cv2

# 2. Input the image 

In [77]:
img = cv2.imread('fogimg.jpg')

# 3. Applying thresholding to segment the image

In [78]:
# Convert the image into grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Perform thresholding segmentation
ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

# Display the segmented image
cv2.imshow('Segmented image using Thresholding', thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 4. Applying Canny Edge Detection Algorithm to segment the image

In [79]:
# Define the edge detection parameters
threshold1 = 30
threshold2 = 100

# Perform Canny edge detection
edges = cv2.Canny(img, threshold1, threshold2)

# Display the edges
cv2.imshow('Segmented image using Canny Edge Detection', edges)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 5. Applying Region Growing to segment the image

In [80]:
# Convert the image into grayscale and remove noise
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

# Function to perform region growing
def region_growing(image, seed):
    out = np.zeros_like(image)
    out[seed] = 255
    queue = [seed]
    while queue:
        pixel = queue.pop(0)
        for neighbor in get_neighbors(pixel, image.shape):
            if abs(image[pixel] - image[neighbor]) < 50 and out[neighbor] == 0:
                out[neighbor] = 255
                queue.append(neighbor)
    return out

# Function to get the neighbours of a given pixel
def get_neighbors(pixel, shape):
    indices = [(pixel[0] - 1, pixel[1] - 1),
               (pixel[0] - 1, pixel[1]),
               (pixel[0] - 1, pixel[1] + 1),
               (pixel[0], pixel[1] - 1),
               (pixel[0], pixel[1] + 1),
               (pixel[0] + 1, pixel[1] - 1),
               (pixel[0] + 1, pixel[1]),
               (pixel[0] + 1, pixel[1] + 1)]
    neighbors = filter(lambda x: 0 <= x[0] < shape[0] and 0 <= x[1] < shape[1], indices)
    return neighbors

# Select the seed point and segment it using region growing
seed = (200, 200)
region_grown = region_growing(gray, seed)

# Display the output image
cv2.imshow("Segmented image using Region Growing", region_grown)
cv2.waitKey(0)
cv2.destroyAllWindows()

<ipython-input-80-5a5db71b2430>:13: RuntimeWarning: overflow encountered in ubyte_scalars
  if abs(image[pixel] - image[neighbor]) < 50 and out[neighbor] == 0:


# 5. Applying Clustering (k-means) to segment the image

In [81]:
# Reshape the input image to a 2D array of pixels
pixel_values = img.reshape((-1, 3))

# Convert the pixel values to float32 for K-means clustering
pixel_values = np.float32(pixel_values)

# Define the parameters for K-means clustering
K = 4 # number of clusters
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

# Perform K-means clustering
ret, labels, centers = cv2.kmeans(pixel_values, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

# Reshape the labels and centers to the original image shape
segmented_image = labels.reshape(img.shape[:2])
centers = np.uint8(centers)
segmented_image = centers[segmented_image]

# Display the segmented image
cv2.imshow('Segmented Image using k-means clustering', segmented_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 6. Applying Erosion and Dilation on the segmented images (Morphological Operations)

In [ ]:
import cv2
import numpy as np

img = segmented_image

# Define kernel size for erosion and dilation
kernel_size = (5,5)

# Create erosion and dilation kernels
erosion_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)
dilation_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)

# Apply erosion to segmented image
eroded_img = cv2.erode(img, erosion_kernel, iterations=1)
# Apply dilation to segmented image
dilated_img = cv2.dilate(img, dilation_kernel, iterations=1)

# Display the original, eroded, and dilated images
cv2.imshow('Eroded Image', eroded_img)
cv2.imshow('Dilated Image', dilated_img)
cv2.waitKey(0)
cv2.destroyAllWindows()